In [1]:
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, utils, models

from hex import board, player, game
import util
import move_evaluator
import move_generator

In [ ]:
games = pd.read_csv("finaldataframe.csv")

In [ ]:
games_over_2 = games[games["move_list"].str.len() > 4]

boards, winners = util.makeInputOutputTotal(games_over_2["move_list"].values[:1000],
                                            games_over_2["winner"].values[:1000])

In [ ]:
p1 = player.ModelPlayer(move_generator.SamplingProbabilisticGenerator(10), move_evaluator.TestMoveEvaluator())
p2 = player.ModelPlayer(move_generator.ExhaustiveProbabilisticGenerator(), move_evaluator.TestMoveEvaluator())
new_moves, new_winners =  util.generate_games(1000, p1, p2)

In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(256, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=(13, 13, 1)),
    layers.Conv2D(256, (3, 3), activation ="relu"),
    layers.MaxPool2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPool2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')])

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(boards.reshape(list(boards.shape) + [1]), winners, epochs=1)


In [ ]:
model.save("blackwins_model.h5")

In [2]:
model = models.load_model("blackwins_model.h5")

In [5]:
p1 = player.ModelPlayer(move_generator.SamplingGenerator(10), move_evaluator.BlackWinMoveEvaluator(model))
p2 = player.ModelPlayer(move_generator.SamplingGenerator(10), move_evaluator.BlackWinMoveEvaluator(model))
new_moves, new_winners = util.generate_games(50, p1, p2)

In [4]:
new_winners

('black', 'black')

In [6]:
wins = pd.Series(new_winners)
len(wins[wins == "black"])

27

In [ ]:
new_boards, new_winners = util.makeInputOutputTotal(new_moves, new_winners, scraped=False)